In [1]:
import mne
from mnelab.io.writers import write_edf
from mff_to_edf import write_edf as rotem_write_edf

In [3]:
noisy = {'013': ['RA2'], '017': [], '018': ['RA1', 'RA2'], '025': ['RA1', 'RA2'], '38': ['RA1'], '39': [], '44': [], '46': [], '47': ['RH1'], '48': ['RAH1', 'LPHG4'], '51': ['RH2', 'LH3']}
crop_start = {'013': 59, '017': 76, '018': 30, '025': 115, '38': 104, '39':113, '44': 250, '46': 20, '47': 76, '48': 8, '51': 25}

In [6]:
# for subj in noisy.keys():
for subj in ['38']:
    # subj = '025'
    edf = f'C:\\UCLA\\P{subj}_overnightData_fix.edf'
    raw = mne.io.read_raw_edf(edf)
    depth = ['RAH', 'LAH', 'RA', 'LA', 'LEC', 'REC', 'RPHG', 'LPHG', 'RMH', 'LMH', 'RH', 'LH']
    not_relevant = ['AC', 'AI', '5', '6', '7', '8', '9', '10']

    to_pick = [x for x in raw.ch_names if any([y in x for y in depth]) and not any([z in x for z in not_relevant])]
    to_pick = [x for x in to_pick if x not in noisy[subj]]
    chans_bi_1 = []
    chans_bi_2 = []
    # get the channels for bipolar reference
    for i, chan in enumerate(to_pick):
        if i + 1 < len(to_pick):
            next_chan = to_pick[i + 1]
            # check that its the same contact
            if next_chan[:-1] == chan[:-1]:
                chans_bi_1.append(chan)
                chans_bi_2.append(next_chan)

    raw.pick_channels(to_pick)
    raw.crop(tmin=crop_start[subj] * 60, tmax=crop_start[subj] * 60 + 180)
    raw.resample(1000)
    raw.load_data()
    raw_bi = mne.set_bipolar_reference(raw, anode=chans_bi_1, cathode=chans_bi_2, drop_refs=False)
    # remove channel 4
    raw_bi.pick_channels([x for x in raw_bi.ch_names if '4' not in x or '-' in x])
    # Maybe another sort?
    # raw_bi.reorder_channels(sorted(raw_bi.ch_names))
    if len(raw_bi.annotations) > 0:
        raw_bi.annotations.delete(list(range(len(raw_bi.annotations))))
    # raw_bi.plot()
    rotem_write_edf(raw_bi, f'D:\\Lilach\\{subj}_for_tag_fix2.edf')

Extracting EDF parameters from C:\UCLA\P38_overnightData_fix.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
EEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=23, n_times=180001
    Range : 6240000 ... 6420000 =   6240.000 ...  6420.000 secs
Ready.
Added the following bipolar channels:
RA2-RA3, RA3-RA4, RAH1-RAH2, RAH2-RAH3, RAH3-RAH4, RMH1-RMH2, RMH2-RMH3, RMH3-RMH4, RPHG1-RPHG2, RPHG2-RPHG3, RPHG3-RPHG4, LA1-LA2, LA2-LA3, LA3-LA4, LAH1-LAH2, LAH2-LAH3, LAH3-LAH4, LMH1-LMH2, LMH2-LMH3, LMH3-LMH4, LPHG1-LPHG2, LPHG2-LPHG3, LPHG3-LPHG4
saving to D:\Lilach\38_for_tag_fix2.edf, filetype 0
conversion time:
0:00:00.269664


In [4]:
# crop stim from tag files
timings = {'485': [0, 2 * 60 + 55], '487': [0, 3 * 60], '489': [0, 3 * 60], '498': [0, 3 * 60], '499': [1 * 60 + 10, 6 * 60], '520': [4 * 60, 6 * 60]}
# for subj in ['485', '487', '489', '498', '499', '520']:
for subj in ['489']:
    raw = mne.io.read_raw_edf(f'D:\\Dawn\\P{subj}_for_tag_edited.edf')
    raw.pick([x for x in raw.ch_names if len(x) > 2])
    raw.crop(tmin=timings[subj][0], tmax=timings[subj][1])
    # fix annotation timing
    if timings[subj][0] > 0:
        raw.annotations.onset -= timings[subj][0]
    rotem_write_edf(raw, f'D:\\final_tags\\P{subj}_for_tag.edf')

Extracting EDF parameters from D:\Dawn\P489_for_tag_edited.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
saving to D:\final_tags\P489_for_tag.edf, filetype 1
conversion time:
0:00:00.460031


In [2]:
timings = {'396': [0, 3 * 60 + 27], '398': [0, 3 * 60 + 14], '402': [0, 3 * 60 + 50], '406': [0, 3 * 60 + 16], '415': [0, 3 * 60 + 21], '416': [0, 3 * 60 + 22]}
for subj in timings.keys():
    raw = mne.io.read_raw_edf(f'D:\\final_tags_with_stim_and_END\\P{subj}_for_tag.edf')
    raw.crop(tmin=timings[subj][0], tmax=timings[subj][1])
    raw.resample(1000)
    rotem_write_edf(raw, f'D:\\final_tags\\P{subj}_for_tag_1000.edf')

Extracting EDF parameters from D:\final_tags_with_stim_and_END\P396_for_tag.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
saving to D:\final_tags\P396_for_tag_1000.edf, filetype 1
conversion time:
0:00:00.192484
Extracting EDF parameters from D:\final_tags_with_stim_and_END\P398_for_tag.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
saving to D:\final_tags\P398_for_tag_1000.edf, filetype 1
conversion time:
0:00:00.151595
Extracting EDF parameters from D:\final_tags_with_stim_and_END\P402_for_tag.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
saving to D:\final_tags\P402_for_tag_1000.edf, filetype 1
conversion time:
0:00:00.132646
Extracting EDF parameters from D:\final_tags_with_stim_and_END\P406_for_tag.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
saving to D:\final_tags\P406_for_tag_1000.edf, filetype 1
conversi

In [6]:
raw.get_channel_types()

['seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg',
 'seeg']

Using qt as 2D backend.
